# Simularium Conversion Tutorial : PhysiCell Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import Converter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium custom data Converter consumes discrete cell data from PhysiCell. 

The Converter requires the following data:
* **box_size** : *np.ndarray (shape = \[3\])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
* **timestep** : *float*
    * A float amount of time that passes each step
* **path_to_output_dir** : *string*
    * A string path to the PhysiCell output directory containing MultiCellDS XML and MATLAB files
    
Optionally, you can also include:
* **types** : *Dict\[int, Dict\[int, str\]\]* (optional)
    * \[cell type ID from PhysiCell data\] : *Dict\[int, str\]*
    * the cell type ID from PhysiCell data mapped to display names for each type of cell
        * \[cell phase ID from PhysiCell data\] : *str*
        * the cell phase ID from PhysiCell data mapped to the display name for this cell type and phase
        * Default: "cell\[cell type ID from PhysiCell data\]#\[cell phase ID from PhysiCell data\]"
* **scale_factor** : *float*
    * A multiplier for the ReaDDy scene, use if visualization is too large or small
    * Default: 1.0
* **plots** : *Dict\[str, Any\]*
    * An object containing plot data already in Simularium format

In [2]:
box_size = 1000.0
path_to_xml_files = "../simulariumio/tests/data/physicell/output/"

example_data = {
    "box_size" : np.array([box_size, box_size, 100.0]),
    "timestep" : 360.0,
    "path_to_output_dir" : path_to_xml_files,
    "scale_factor" : 0.01,
}

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [3]:
Converter(example_data, "physicell").write_JSON("/Users/blairl/Desktop/physicell_example")

Reading ../simulariumio/tests/data/physicell/output/output00000000.xml
Reading ../simulariumio/tests/data/physicell/output/output00000000_cells_physicell.mat
Reading ../simulariumio/tests/data/physicell/output/output00000001.xml
Reading ../simulariumio/tests/data/physicell/output/output00000001_cells_physicell.mat
Reading ../simulariumio/tests/data/physicell/output/output00000002.xml
Reading ../simulariumio/tests/data/physicell/output/output00000002_cells_physicell.mat


## Visualize in the Simularium viewer

In your browser, either Firefox or Chrome, navigate to https://staging.agentviz.allencell.org/ and drag your file onto the center viewer window. 

*For now you'll first have to choose an example trajectory and close the load window. Once the example trajectory loads, you can drop your own file in to replace it. We'll fix this soon :)*